In [3]:
from __future__ import print_function
2
import cv2
3
import numpy as np
4
 
5
 
6
MAX_FEATURES = 500
7
GOOD_MATCH_PERCENT = 0.15
8
 
9
 
10
def alignImages(im1, im2):

  # Convert images to grayscale

  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
14
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
15
   
16
  # Detect ORB features and compute descriptors.
17
  orb = cv2.ORB_create(MAX_FEATURES)
18
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
19
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
20
   
21
  # Match features.
22
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
23
  matches = matcher.match(descriptors1, descriptors2, None)
24
   
25
  # Sort matches by score
26
  matches.sort(key=lambda x: x.distance, reverse=False)
27
 
28
  # Remove not so good matches
29
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
30
  matches = matches[:numGoodMatches]
31
 
32
  # Draw top matches
33
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
34
  cv2.imwrite("matches.jpg", imMatches)
35
   
36
  # Extract location of good matches
37
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
38
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
39
 
40
  for i, match in enumerate(matches):
41
    points1[i, :] = keypoints1[match.queryIdx].pt
42
    points2[i, :] = keypoints2[match.trainIdx].pt
43
   
44
  # Find homography
45
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
46
 
47
  # Use homography
48
  height, width, channels = im2.shape
49
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
50
   
51
  return im1Reg, h
52
 
53
 
54
if __name__ == '__main__':
55
   
56
  # Read reference image
57
  refFilename = "form.jpg"
58
  print("Reading reference image : ", refFilename)
59
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
60
 
61
  # Read image to be aligned
62
  imFilename = "scanned-form.jpg"
63
  print("Reading image to align : ", imFilename); 
64
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
65
   
66
  print("Aligning images ...")
67
  # Registered image will be resotred in imReg.
68
  # The estimated homography will be stored in h.
69
  imReg, h = alignImages(im, imReference)
70
   
71
  # Write aligned image to disk.
72
  outFilename = "aligned.jpg"
73
  print("Saving aligned image : ", outFilename);
74
  cv2.imwrite(outFilename, imReg)
75
 
76
  # Print estimated homography
77
  print("Estimated homography : \n",  h)


IndentationError: unexpected indent (<ipython-input-3-bb2fbf91ebe8>, line 25)